In [ ]:
t0 = time.time()

# UPLOAD DATA 

conn        = connectivity.Connectivity.from_file()

weight_symm = [...]

np.savez(out_weight,mask_inter=mask_inter,mask_frontal=mask_frontal,weights_orig=weights_orig)

#DEFINE THE MODEL

conn               = connectivity.Connectivity(
    weights=weights_symm,
    region_labels=np.array(np.zeros(np.shape(weights_symm)[0]),dtype='<U128'),
    tract_lengths=np.zeros(np.shape(weights_symm)),
    areas =np.zeros(np.shape(weights_symm)[0]),
    speed=np.array(np.Inf,dtype=np.float),
    centres = np.zeros(np.shape(weights_symm)[0])) # default 76 regions 
# conn_speed         = np.Inf  
# conn.weights       = weights_symm
# conn.areas         = np.zeros(np.shape(weights_symm))
# conn.tract_lengths = np.zeros(np.shape(weights_symm))
print('weight:',weights_symm) 
print('shape :',np.shape(conn.weights))
print('conn :', conn.weights)

mpr=models.MontbrioPazoRoxin(
        eta   = np.r_[-4.6],
        J     = np.r_[14.5],
        Delta = np.r_[0.7],
        tau   = np.r_[1],
    )
#     mpr.state_variable_range['r'] = np.array([0.,.25])

sim = simulator.Simulator(model=mpr,
connectivity=conn,
    coupling=coupling.Scaling(
      a=np.r_[G]
    ),
conduction_speed=np.Inf,
    integrator=integrators.HeunStochastic(
      dt=dt,
      noise=noise.Additive(
          nsig=np.r_[nsigma, nsigma*2],
          noise_seed=seed
      )
),
monitors=[
      monitors.TemporalAverage(period=0.1),
    ]
)

sim.configure()

(TemporalAverage_time, TemporalAverage_data), = simulation.run_nbMPR_backend(sim, simulation_length=sim_len)
TemporalAverage_time *= 10 # rescale time
Bold_time, Bold_data = simulation.tavg_to_bold(TemporalAverage_time, TemporalAverage_data, tavg_period=1., connectivity=sim.connectivity, svar=0, decimate=2000)

#     np.savez(out_path, TemporalAverage_data = TemporalAverage_data, TemporalAverage_time = TemporalAverage_time)

np.savez(out_path, Bold_data = Bold_data, Bold_time = Bold_time,TemporalAverage_data = TemporalAverage_data, TemporalAverage_time = TemporalAverage_time)

CPU_TIME = time.time() - t0
print(['CPU time-->',CPU_TIME])

